In [ ]:
# install packages in libs (as editable)
# !python -m pip install huggingface_hub==0.21.4
# !python -m pip install safetensors==0.4.2
# !python -m pip install -r ../requirements.txt
# !python -m pip install -e ../libs

In [2]:
from huggingface_hub import hf_hub_download
from safetensors import safe_open

import json, torch

cfg_file_path = hf_hub_download(repo_id="google/owlv2-base-patch16-ensemble", filename="config.json")
print('config.json', cfg_file_path)

model_file_path = hf_hub_download(repo_id="google/owlv2-base-patch16-ensemble", filename="model.safetensors")
print('model.safetensors', model_file_path)

preproc_file_path = hf_hub_download(repo_id="google/owlv2-base-patch16-ensemble", filename="preprocessor_config.json")
print('preprocessor_config.json', preproc_file_path)

state_dict = {}
with safe_open(model_file_path, framework="pt", device="cpu") as f:
   for key in f.keys():
       state_dict[key] = f.get_tensor(key)
print(state_dict.keys())

with open(cfg_file_path, 'r') as f:
    config = json.load(f)
# print(json.dumps(config, indent=2))

with open(preproc_file_path, 'r') as f:
    preproc_config = json.load(f)
# print(json.dumps(preproc_config, indent=2))


config.json /Users/n0c09jf/.cache/huggingface/hub/models--google--owlv2-base-patch16-ensemble/snapshots/7c77fc1316a353da3109f81d80ae118dd58fd55b/config.json
model.safetensors /Users/n0c09jf/.cache/huggingface/hub/models--google--owlv2-base-patch16-ensemble/snapshots/7c77fc1316a353da3109f81d80ae118dd58fd55b/model.safetensors
preprocessor_config.json /Users/n0c09jf/.cache/huggingface/hub/models--google--owlv2-base-patch16-ensemble/snapshots/7c77fc1316a353da3109f81d80ae118dd58fd55b/preprocessor_config.json
dict_keys(['box_head.dense0.bias', 'box_head.dense0.weight', 'box_head.dense1.bias', 'box_head.dense1.weight', 'box_head.dense2.bias', 'box_head.dense2.weight', 'class_head.dense0.bias', 'class_head.dense0.weight', 'class_head.logit_scale.bias', 'class_head.logit_scale.weight', 'class_head.logit_shift.bias', 'class_head.logit_shift.weight', 'layer_norm.bias', 'layer_norm.weight', 'objectness_head.dense0.bias', 'objectness_head.dense0.weight', 'objectness_head.dense1.bias', 'objectness_h

In [4]:
from binsense.owlv2 import Owlv2ForObjectDetection
from binsense.owlv2.config import Owlv2Config
from binsense.owlv2 import Owlv2ImageProcessor

owl_processor = Owlv2ImageProcessor(**preproc_config)
owl_config = Owlv2Config(**config)

model = Owlv2ForObjectDetection(owl_config)
model_module_keys = model.__dict__['_modules'].keys()
appl_state_dict = dict()
for key in state_dict.keys():
    new_key = key
    first_token = key.split('.')[0]
    if first_token == 'owlv2':
        new_key = '.'.join(key.split('.')[1:])
        
    if new_key.split('.')[0] in model_module_keys:
        appl_state_dict[new_key] = state_dict[key]

model.load_state_dict(appl_state_dict)
print(model)

Owlv2ForObjectDetection(
  (vision_model): Owlv2VisionTransformer(
    (embeddings): Owlv2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
      (position_embedding): Embedding(3601, 768)
    )
    (pre_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): Owlv2Encoder(
      (layers): ModuleList(
        (0-11): 12 x Owlv2EncoderLayer(
          (self_attn): Owlv2Attention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Owlv2MLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bi